```
--- Day 23: Safe Cracking ---

This is one of the top floors of the nicest tower in EBHQ. The Easter Bunny's private office is here, complete with a safe hidden behind a painting, and who wouldn't hide a star in a safe behind a painting?

The safe has a digital screen and keypad for code entry. A sticky note attached to the safe has a password hint on it: "eggs". The painting is of a large rabbit coloring some eggs. You see 7.

When you go to type the code, though, nothing appears on the display; instead, the keypad comes apart in your hands, apparently having been smashed. Behind it is some kind of socket - one that matches a connector in your prototype computer! You pull apart the smashed keypad and extract the logic circuit, plug it into your computer, and plug your computer into the safe.

Now, you just need to figure out what output the keypad would have sent to the safe. You extract the assembunny code from the logic chip (your puzzle input).
The code looks like it uses almost the same architecture and instruction set that the monorail computer used! You should be able to use the same assembunny interpreter for this as you did there, but with one new instruction:

tgl x toggles the instruction x away (pointing at instructions like jnz does: positive means forward; negative means backward):

For one-argument instructions, inc becomes dec, and all other one-argument instructions become inc.
For two-argument instructions, jnz becomes cpy, and all other two-instructions become jnz.
The arguments of a toggled instruction are not affected.
If an attempt is made to toggle an instruction outside the program, nothing happens.
If toggling produces an invalid instruction (like cpy 1 2) and an attempt is later made to execute that instruction, skip it instead.
If tgl toggles itself (for example, if a is 0, tgl a would target itself and become inc a), the resulting instruction is not executed until the next time it is reached.
For example, given this program:

cpy 2 a
tgl a
tgl a
tgl a
cpy 1 a
dec a
dec a
cpy 2 a initializes register a to 2.
The first tgl a toggles an instruction a (2) away from it, which changes the third tgl a into inc a.
The second tgl a also modifies an instruction 2 away from it, which changes the cpy 1 a into jnz 1 a.
The fourth line, which is now inc a, increments a to 3.
Finally, the fifth line, which is now jnz 1 a, jumps a (3) instructions ahead, skipping the dec a instructions.
In this example, the final value in register a is 3.

The rest of the electronics seem to place the keypad entry (the number of eggs, 7) in register a, run the code, and then send the value left in register a to the safe.

What value should be sent to the safe?
```

In [22]:
class BunnyMachine:
        
    def __init__(self, a=0, b=0, c=0, d=0):
        self.registers = { 'a': a, 'b': b, 'c': c, 'd': d}
        self.op = 0
        self.toggle = set()
        self.ops = { 'cpy': self.cpy, 'inc': self.inc, 'dec': self.dec, 'jnz': self.jnz, 'tgl': self.tgl }
    
    def execute(self, program):
        
        # Compile the program
        bytecode = []
        for instruction in program:
            params = instruction.split()
            method = self.ops[params[0]]
            bytecode.append((method, params[1:]))
            
        # Execute the program
        self.op = 0
        while self.op < len(bytecode):
            method, params = bytecode[self.op]
            
            if self.op in self.toggle:
                if len(params) == 1:
                    method = self.ops['dec'] if method == self.ops['inc'] else self.ops['inc']
                elif len(params) == 2:
                    method = self.ops['cpy'] if method == self.ops['jnz'] else self.ops['jnz']
            
            res = method(*params)
            self.op += 1 if res is None else res
        
        print(self.registers)
        
    def val(self, x):
        return self.registers[x] if x in self.registers else int(x)
    
    def tgl(self, x):
        self.toggle.add(self.op + self.val(x))
    
    def cpy(self, x, y):
        self.registers[y] = self.val(x)
        
    def inc(self, x):
        self.registers[x]+=1
        
    def dec(self, x):
        self.registers[x]-=1
        
    def jnz(self, x, y):
        return None if self.val(x) == 0 else self.val(y)   

In [23]:
example = [
    "cpy 2 a",
    "tgl a",
    "tgl a",
    "tgl a",
    "cpy 1 a",
    "dec a",
    "dec a"
]

In [24]:
BunnyMachine().execute(example)

{'a': 3, 'c': 0, 'd': 0, 'b': 0}


In [27]:
%%time
with open('inputs/day23.txt') as fd:
    BunnyMachine(a=7).execute(fd)

{'a': 13140, 'c': 0, 'd': 0, 'b': 1}
CPU times: user 76 ms, sys: 0 ns, total: 76 ms
Wall time: 74 ms


```
--- Part Two ---

The safe doesn't open, but it does make several angry noises to express its frustration.

You're quite sure your logic is working correctly, so the only other thing is... you check the painting again. As it turns out, colored eggs are still eggs. Now you count 12.

As you run the program with this new input, the prototype computer begins to overheat. You wonder what's taking so long, and whether the lack of any instruction more powerful than "add one" has anything to do with it. Don't bunnies usually multiply?

Anyway, what value should actually be sent to the safe?
```

In [28]:
%%time
with open('inputs/day23.txt') as fd:
    BunnyMachine(a=12).execute(fd)

{'a': 479009700, 'c': 0, 'd': 0, 'b': 1}
CPU times: user 49min 38s, sys: 344 ms, total: 49min 38s
Wall time: 49min 39s
